In [6]:
!pip install autogen[kubernetes_executor]@git+https://github.com/fere-ai/autogen.git@a49

  Cloning https://github.com/fere-ai/autogen.git (to revision a49) to /tmp/pip-install-33yd_2r6/autogen_605fc08bd46647e4a29ea463fecc207a
  Running command git clone --filter=blob:none --quiet https://github.com/fere-ai/autogen.git /tmp/pip-install-33yd_2r6/autogen_605fc08bd46647e4a29ea463fecc207a
  Running command git checkout -b a49 --track origin/a49
  Switched to a new branch 'a49'
  branch 'a49' set up to track 'origin/a49'.
  Resolved https://github.com/fere-ai/autogen.git to commit e1e27c0cc498e30ce9841e02330f55cd19dc3d09
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
from autogen.coding.kubernetes_commandline_code_executor import KubernetesCommandLineCodeExecutor

In [8]:
executor = KubernetesCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir="/workspace",  # Use the temporary directory to store the code files.
)

The above code creates a pod in your kubernetes cluster. You can check that by

```
$ kubectl get pods
NAME                                                     READY   STATUS    RESTARTS   AGE
autogen-code-exec-8a0961b1-d9e5-42fb-94e4-24bc69bc442b   1/1     Running   0          115s
```

In [10]:
from autogen.agentchat import ConversableAgent

code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

In [13]:
message_with_code_block = """This is a message with code block.
The code block is below:
```python
print("hello world")
```
This is the end of the message.
"""

In [14]:
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: hello world


Stop the kubernetes command line executor to clean up the pods.

In [15]:
executor.stop()  # Stop